<a href="https://colab.research.google.com/github/Dhananjay-97/notebooks/blob/main/gemini_crewai_react_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Support Analysis with Gemini 2.5 Pro and CrewAI

CrewAI is a framework for orchestrating autonomous AI agents that collaborate to achieve complex goals. It lets you define agents by specifying roles, goals, and backstories, and then define tasks for them.

This example demonstrates how to build a multi-agent system for analyzing customer support data to identify issues and propose process improvements using Gemini 2.5 Pro, generating a report intended to be read by a Chief Operating Officer (COO).

The guide will show you how to create a "crew" of AI agents that can do the following tasks:

1.   Fetch and analyze customer support data (simulated in this example).
2.   Identify recurring problems and process bottlenecks.
3.   Suggest actionable improvements.
4.   Compile the findings into a concise report suitable for a COO.




In [1]:
! pip install "crewai[tools]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.5/620.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.1/327.1 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.7 MB/s e

Set your Gemini API key as an environment variable named GEMINI_API_KEY, then configure CrewAI to use the Gemini 2.5 Pro model.

In [8]:
import os
from crewai import LLM

# Read your API key from the environment variable
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

# Use Gemini 2.5 Pro Experimental model
gemini_llm = LLM(
    model='gemini/gemini-2.5-pro',
    api_key=gemini_api_key,
    temperature=0.0  # Lower temperature for more consistent results.
)

# Define components

CrewAI applications are built using Tools, Agents, Tasks, and the Crew itself. Each of these is explained in the following sections.

# Tools
Tools are capabilities that agents can use to interact with the outside world or perform specific actions. Here, you define a placeholder tool to simulate fetching customer support data. In a real application, you would connect to a database, API or file system. For more information on tools, see the CrewAI tools guide.

In [9]:
from crewai.tools import BaseTool

# Placeholder tool for fetching customer support data
class CustomerSupportDataTool(BaseTool):
    name: str = "Customer Support Data Fetcher"
    description: str = (
      "Fetches recent customer support interactions, tickets, and feedback. "
      "Returns a summary string.")

    def _run(self, argument: str) -> str:
        # In a real scenario, this would query a database or API.
        # For this example, return simulated data.
        print(f"--- Fetching data for query: {argument} ---")
        return (
            """Recent Support Data Summary:
- 50 tickets related to 'login issues'. High resolution time (avg 48h).
- 30 tickets about 'billing discrepancies'. Mostly resolved within 12h.
- 20 tickets on 'feature requests'. Often closed without resolution.
- Frequent feedback mentions 'confusing user interface' for password reset.
- High volume of calls related to 'account verification process'.
- Sentiment analysis shows growing frustration with 'login issues' resolution time.
- Support agent notes indicate difficulty reproducing 'login issues'."""
        )

support_data_tool = CustomerSupportDataTool()

# Agents
Agents are the individual AI workers in your crew. Each agent has a specific role, goal, backstory, assigned llm, and optional tools. For more information on agents, see the CrewAI agents guide.

In [10]:
from crewai import Agent

# Agent 1: Data analyst
data_analyst = Agent(
    role='Customer Support Data Analyst',
    goal='Analyze customer support data to identify trends, recurring issues, and key pain points.',
    backstory=(
        """You are an expert data analyst specializing in customer support operations.
        Your strength lies in identifying patterns and quantifying problems from raw support data."""
    ),
    verbose=True,
    allow_delegation=False,  # This agent focuses on its specific task
    tools=[support_data_tool],  # Assign the data fetching tool
    llm=gemini_llm  # Use the configured Gemini LLM
)

# Agent 2: Process optimizer
process_optimizer = Agent(
    role='Process Optimization Specialist',
    goal='Identify bottlenecks and inefficiencies in current support processes based on the data analysis. Propose actionable improvements.',
    backstory=(
        """You are a specialist in optimizing business processes, particularly in customer support.
        You excel at pinpointing root causes of delays and inefficiencies and suggesting concrete solutions."""
    ),
    verbose=True,
    allow_delegation=False,
    # No tools needed, this agent relies on the context provided by data_analyst.
    llm=gemini_llm
)

# Agent 3: Report writer
report_writer = Agent(
    role='Executive Report Writer',
    goal='Compile the analysis and improvement suggestions into a concise, clear, and actionable report for the COO.',
    backstory=(
        """You are a skilled writer adept at creating executive summaries and reports.
        You focus on clarity, conciseness, and highlighting the most critical information and recommendations for senior leadership."""
    ),
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm
)

# Tasks
Tasks define the specific assignments for the agents. Each task has a description, expected_output, and is assigned to an agent. Tasks are run sequentially by default and include the context of the previous task. For more information on tasks, see the CrewAI tasks guide.

In [11]:
from crewai import Task

# Task 1: Analyze data
analysis_task = Task(
    description=(
        """Fetch and analyze the latest customer support interaction data (tickets, feedback, call logs)
        focusing on the last quarter. Identify the top 3-5 recurring issues, quantify their frequency
        and impact (e.g., resolution time, customer sentiment). Use the Customer Support Data Fetcher tool."""
    ),
    expected_output=(
        """A summary report detailing the key findings from the customer support data analysis, including:
- Top 3-5 recurring issues with frequency.
- Average resolution times for these issues.
- Key customer pain points mentioned in feedback.
- Any notable trends in sentiment or support agent observations."""
    ),
    agent=data_analyst  # Assign task to the data_analyst agent
)

# Task 2: Identify bottlenecks and suggest improvements
optimization_task = Task(
    description=(
        """Based on the data analysis report provided by the Data Analyst, identify the primary bottlenecks
        in the support processes contributing to the identified issues (especially the top recurring ones).
        Propose 2-3 concrete, actionable process improvements to address these bottlenecks.
        Consider potential impact and ease of implementation."""
    ),
    expected_output=(
        """A concise list identifying the main process bottlenecks (e.g., lack of documentation for agents,
        complex escalation path, UI issues) linked to the key problems.
A list of 2-3 specific, actionable recommendations for process improvement
(e.g., update agent knowledge base, simplify password reset UI, implement proactive monitoring)."""
    ),
    agent=process_optimizer  # Assign task to the process_optimizer agent
    # This task implicitly uses the output of analysis_task as context
)

# Task 3: Compile COO report
report_task = Task(
    description=(
        """Compile the findings from the Data Analyst and the recommendations from the Process Optimization Specialist
        into a single, concise executive report for the COO. The report should clearly state:
1. The most critical customer support issues identified (with brief data points).
2. The key process bottlenecks causing these issues.
3. The recommended process improvements.
Ensure the report is easy to understand, focuses on actionable insights, and is formatted professionally."""
    ),
    expected_output=(
        """A well-structured executive report (max 1 page) summarizing the critical support issues,
        underlying process bottlenecks, and clear, actionable recommendations for the COO.
        Use clear headings and bullet points."""
    ),
    agent=report_writer  # Assign task to the report_writer agent
)

# Crew
The Crew brings the agents and tasks together, defining the workflow process (such as "sequential").

In [12]:
from crewai import Crew, Process

# Define the crew with agents, tasks, and process
support_analysis_crew = Crew(
    agents=[data_analyst, process_optimizer, report_writer],
    tasks=[analysis_task, optimization_task, report_task],
    process=Process.sequential,  # Tasks will run sequentially in the order defined
    verbose=True
)

# Run the Crew
Finally, kick off the crew execution with any necessary inputs.

In [13]:
# Start the crew's work
print("--- Starting Customer Support Analysis Crew ---")
# The 'inputs' dictionary provides initial context if needed by the first task.
# In this case, the tool simulates data fetching regardless of the input.
result = support_analysis_crew.kickoff(inputs={'data_query': 'last quarter support data'})

print("--- Crew Execution Finished ---")
print("--- Final Report for COO ---")
print(result)

--- Starting Customer Support Analysis Crew ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: edd3e203-cd1e-43db-839e-c0653388cd7f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Support Data Analyst                                                                           │
│                                                                                                                 │
│  Task: Fetch and analyze the latest customer support interaction data (tickets, feedback, call logs)            │
│          focusing on the last quarter. Identify the top 3-5 recurring issues, quantify their frequency          │
│          and impact (e.g., resolution time, customer sentiment). Use the Customer Support Data Fetcher tool.    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

--- Fetching data for query: Fetch and summarize all customer support interactions, tickets, and feedback from the 
last quarter. ---

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Executive Report Writer                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **TO:** Chief Operating Officer                                                                                │
│  **FROM:** Executive Report Writer                                                                              │
│  **DATE:** October 26, 2023                                                                                     │
│  **SUBJECT:** Executive Summary: Analysis and Recommendations for Customer Support Process Improvement          │
│                                                                                                                 │
│  ### 1. Executive Summary                                                                                       │
│                                                                                                                 │
│  This report summarizes an analysis of last quarter's customer support data to identify critical operational    │
│  issues and recommend targeted process improvements. The analysis reveals that **Login Issues** are the         │
│  primary driver of customer dissatisfaction, characterized by a high volume of support tickets and an           │
│  unacceptable **48-hour average resolution time**. These delays are caused by poor agent diagnostic             │
│  capabilities, a confusing user self-service experience, and the lack of a formal escalation process for        │
│  critical access problems. The following recommendations are designed to directly address these bottlenecks to  │
│  significantly reduce resolution times, decrease support ticket volume, and improve overall customer            │
│  satisfaction.                                                                                                  │
│                                                                                                                 │
│  ### 2. Critical Issues Identified (Last Quarter Data)                                                          │
│                                                                                                                 │
│  Our analysis of customer support data has pinpointed several key performance gaps that require immediate       │
│  attention.                                                                                                     │
│                                                                                                                 │
│  *   **Critical Failure in Account Access:** "Login Issues" are the most frequent problem, accounting for **50  │
│  tickets** last quarter. The **48-hour average resolution time** for this issue is the single largest source    │
│  of customer frustration and negative sentiment.                                                                │
│  *   **High-Friction User Processes:** A high, unquantified volume of support calls is related to the           │
│  "Difficult Account Verification" process. This, combined with a "Confusing Password Reset UI," indicates that  │
│  our self-service tools are creating support demand rather than deflecting it.                                  │
│  *   **Growing Customer Dissatisfaction:** Sentiment analysis shows a direct correlation between the 48-hour    │
│  login resolution time and increasing customer frustration, posing a significant risk to customer retention.    │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 45cefda9-bc62-4f3f-a915-88183e0aee27                                                                     │
│  Agent: Executive Report Writer                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: edd3e203-cd1e-43db-839e-c0653388cd7f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **TO:** Chief Operating Officer                                                                  │
│  **FROM:** Executive Report Writer                                                                              │
│  **DATE:** October 26, 2023                                                                                     │
│  **SUBJECT:** Executive Summary: Analysis and Recommendations for Customer Support Process Improvement          │
│                                                                                                                 │
│  ### 1. Executive Summary                                                                                       │
│                                                                                                                 │
│  This report summarizes an analysis of last quarter's customer support data to identify critical operational    │
│  issues and recommend targeted process improvements. The analysis reveals that **Login Issues** are the         │
│  primary driver of customer dissatisfaction, characterized by a high volume of support tickets and an           │
│  unacceptable **48-hour average resolution time**. These delays are caused by poor agent diagnostic             │
│  capabilities, a confusing user self-service experience, and the lack of a formal escalation process for        │
│  critical access problems. The following recommendations are designed to directly address these bottlenecks to  │
│  significantly reduce resolution times, decrease support ticket volume, and improve overall customer            │
│  satisfaction.                                                                                                  │
│                                                                                                                 │
│  ### 2. Critical Issues Identified (Last Quarter Data)                                                          │
│                                                                                                                 │
│  Our analysis of customer support data has pinpointed several key performance gaps that require immediate       │
│  attention.                                                                                                     │
│                                                                                                                 │
│  *   **Critical Failure in Account Access:** "Login Issues" are the most frequent problem, accounting for **50  │
│  tickets** last quarter. The **48-hour average resolution time** for this issue is the single largest source    │
│  of customer frustration and negative sentiment.                                                                │
│  *   **High-Friction User Processes:** A high, unquantified volume of support calls is related to the           │
│  "Difficult Account Verification" process. This, combined with a "Confusing Password Reset UI," indicates that  │
│  our self-service tools are creating support demand rather than deflecting it.                                  │
│  *   **Growing Customer Dissatisfaction:** Sentiment analysis shows a direct correlation between the 48-hour    │
│  login resolution time and increasing customer frustra

--- Crew Execution Finished ---
--- Final Report for COO ---
**TO:** Chief Operating Officer
**FROM:** Executive Report Writer
**DATE:** October 26, 2023
**SUBJECT:** Executive Summary: Analysis and Recommendations for Customer Support Process Improvement

### 1. Executive Summary

This report summarizes an analysis of last quarter's customer support data to identify critical operational issues and recommend targeted process improvements. The analysis reveals that **Login Issues** are the primary driver of customer dissatisfaction, characterized by a high volume of support tickets and an unacceptable **48-hour average resolution time**. These delays are caused by poor agent diagnostic capabilities, a confusing user self-service experience, and the lack of a formal escalation process for critical access problems. The following recommendations are designed to directly address these bottlenecks to significantly reduce resolution times, decrease support ticket volume, and improve overall c

The script will now execute. The Data Analyst will use the tool, the Process Optimizer will analyze the findings, and the Report Writer will compile the final report, which is then printed to the console. The verbose=True setting will show the detailed thought process and actions of each agent.